### Fonctions analytiques  
Effectuez des calculs complexes sur des groupes de lignes.  

#### Introduction  
Dans le micro-cours d'introduction à SQL, vous avez appris à utiliser les fonctions d'agrégation, qui effectuent des calculs sur des ensembles de lignes. Dans ce tutoriel, vous allez apprendre à définir des **fonctions analytiques**, qui opèrent également sur un ensemble de lignes. Cependant, contrairement aux fonctions d'agrégation, les fonctions analytiques renvoient une valeur (potentiellement différente) pour chaque ligne de la table d'origine.  

Les fonctions analytiques nous permettent d'effectuer des calculs complexes avec une syntaxe relativement simple. Par exemple, nous pouvons rapidement calculer des moyennes mobiles et des totaux cumulés, entre autres quantités.  

#### Syntaxe  
Pour comprendre comment écrire des fonctions analytiques, nous allons travailler avec une petite table contenant des données de deux personnes différentes qui s'entraînent pour une course. La colonne `id` identifie chaque coureur, la colonne `date` contient le jour de la session d'entraînement, et `time` montre le temps (en minutes) que le coureur a consacré à l'entraînement. **Supposons que nous voulions calculer une moyenne mobile des temps d'entraînement** pour chaque coureur, où nous prenons toujours la moyenne de la session actuelle et de la session précédente. Nous pouvons le faire avec la requête suivante :  

```sql
first_query
```

Toutes les fonctions analytiques ont une **clause** `OVER`, qui définit les ensembles de lignes utilisés dans chaque calcul. La clause `OVER` a trois parties (optionnelles) :  

1. **La clause `PARTITION BY`** divise les lignes de la table en différents groupes. Dans la requête ci-dessus, nous divisons par `id` afin que les calculs soient séparés par coureur.  
2. **La clause `ORDER BY`** définit un ordre au sein de chaque partition. Dans l'exemple de requête, l'ordre par la colonne `date` garantit que les sessions d'entraînement antérieures apparaissent en premier.  
3. **La clause finale (`ROWS BETWEEN 1 PRECEDING AND CURRENT ROW`)** est connue sous le nom de clause de cadre de fenêtre. Elle identifie l'ensemble des lignes utilisées dans chaque calcul. Nous pouvons désigner ce groupe de lignes comme une fenêtre. (En fait, les fonctions analytiques sont parfois appelées **fonctions de fenêtre analytiques** ou **simplement fonctions de fenêtre** !)  

```sql
first_query
```

#### (Plus d'informations sur) les clauses de cadre de fenêtre  
Il existe plusieurs façons d'écrire des clauses de cadre de fenêtre :  

- `ROWS BETWEEN 1 PRECEDING AND CURRENT ROW` : la ligne précédente et la ligne actuelle.  
- `ROWS BETWEEN 3 PRECEDING AND 1 FOLLOWING` : les 3 lignes précédentes, la ligne actuelle et la ligne suivante.  
- `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` : toutes les lignes de la partition.  

Bien sûr, cette liste n'est pas exhaustive, et vous pouvez imaginer qu'il existe de nombreuses autres options ! Dans le code ci-dessous, vous verrez certaines de ces clauses en action.  

#### Trois types de fonctions analytiques  
L'exemple ci-dessus utilise une seule des nombreuses fonctions analytiques. **BigQuery** prend en charge une grande variété de fonctions analytiques, et nous en explorerons quelques-unes ici. Pour une liste complète, vous pouvez consulter la documentation.  

1) **Fonctions d'agrégation analytiques**  
Comme vous vous en souvenez peut-être, `AVG()` (de l'exemple ci-dessus) est une fonction d'agrégation. La clause `OVER` est ce qui garantit qu'elle est traitée comme une fonction d'agrégation analytique. Les fonctions d'agrégation prennent toutes les valeurs de la fenêtre comme entrée et renvoient une seule valeur.  

- `MIN()` (ou `MAX()`) : Renvoie la valeur minimale (ou maximale) des valeurs d'entrée.  
- `AVG()` (ou `SUM()`) : Renvoie la moyenne (ou la somme) des valeurs d'entrée.  
- `COUNT()` : Renvoie le nombre de lignes dans l'entrée.  

2) **Fonctions de navigation analytiques**  
Les fonctions de navigation attribuent une valeur en fonction de la valeur d'une ligne (généralement) différente de la ligne actuelle.  

- `FIRST_VALUE()` (ou `LAST_VALUE()`) : Renvoie la première (ou la dernière) valeur de l'entrée.  
- `LEAD()` (et `LAG()`) : Renvoie la valeur d'une ligne suivante (ou précédente).  

3) **Fonctions de numérotation analytiques**  
Les fonctions de numérotation attribuent des valeurs entières à chaque ligne en fonction de l'ordre.  

- `ROW_NUMBER()` : Renvoie l'ordre dans lequel les lignes apparaissent dans l'entrée (en commençant par 1).  
- `RANK()` : Toutes les lignes avec la même valeur dans la colonne de tri reçoivent la même valeur de rang, où la ligne suivante reçoit une valeur de rang qui s'incrémente du nombre de lignes avec le rang précédent. 

---
#### Exemple  
Nous allons travailler avec le jeu de données **San Francisco Open Data**. Nous commençons par examiner les premières lignes de la table `bikeshare_trips`.


In [3]:
from google.cloud import bigquery

# Créer un objet "Client"
client = bigquery.Client()

# Construire une référence au dataset "san_francisco"
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")

# Requête API - récupérer le dataset
dataset = client.get_dataset(dataset_ref)

# Construire une référence à la table "bikeshare_trips"
table_ref = dataset_ref.table("bikeshare_trips")

# Requête API - récupérer la table
table = client.get_table(table_ref)

# Aperçu des cinq premières lignes de la table
client.list_rows(table, max_results=5).to_dataframe()

trip_id  duration_sec                start_date  \
0  1235850          1540 2016-06-11 08:19:00+00:00   
1  1219337          6324 2016-05-29 12:49:00+00:00   
2   793762        115572 2015-06-04 09:22:00+00:00   
3   453845         54120 2014-09-15 16:53:00+00:00   
4  1245113          5018 2016-06-17 20:08:00+00:00   

                  start_station_name  start_station_id  \
0  San Jose Diridon Caltrain Station                 2   
1  San Jose Diridon Caltrain Station                 2   
2  San Jose Diridon Caltrain Station                 2   
3  San Jose Diridon Caltrain Station                 2   
4  San Jose Diridon Caltrain Station                 2   

                   end_date                   end_station_name  \
0 2016-06-11 08:45:00+00:00  San Jose Diridon Caltrain Station   
1 2016-05-29 14:34:00+00:00  San Jose Diridon Caltrain Station   
2 2015-06-05 17:28:00+00:00  San Jose Diridon Caltrain Station   
3 2014-09-16 07:55:00+00:00  San Jose Diridon Caltrain Station   
4 2016-06-17 21:32:00+00:00  San Jose Diridon Caltrain Station   

   end_station_id  bike_number zip_code subscriber_type  
0               2          124    15206        Customer  
1               2          174    55416        Customer  
2               2          190    95391        Customer  
3               2          127       81        Customer  
4               2          153    95070        Customer

**Chaque ligne de la table correspond à un trajet de vélo différent**, et nous pouvons utiliser **une fonction analytique** pour calculer le nombre cumulé de trajets pour chaque date en **2015**.  

In [8]:
# Requête pour compter le nombre (cumulé) de trajets par jour
num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                      COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                               ) AS cumulative_trips
                      FROM trips_by_day
                  """

# Exécuter la requête et renvoyer un DataFrame pandas
num_trips_result = client.query(num_trips_query).result().to_dataframe()
num_trips_result.head()

trip_date  num_trips  cumulative_trips
0  2015-10-17        292            290485
1  2015-03-13       1208             67569
2  2015-01-14       1296             12005
3  2015-04-13       1337             98690
4  2015-05-23        392            140442

La requête utilise une expression de table commune (**CTE**) pour d'abord calculer le nombre quotidien de trajets. Ensuite, nous utilisons `SUM()` comme fonction d'agrégation.  

- Puisqu'il n'y a pas de clause `PARTITION BY`, toute la table est traitée comme une seule partition.  
- La clause `ORDER BY` ordonne les lignes par date, où les dates antérieures apparaissent en premier.  
- En définissant la clause de cadre de fenêtre sur `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`, nous nous assurons que toutes les lignes jusqu'à et y compris la date actuelle sont utilisées pour calculer la somme (cumulée). (Remarque : Si vous lisez la documentation, vous verrez que c'est le comportement par défaut, donc la requête renverrait le même résultat si nous omettions cette clause de cadre de fenêtre.)  

---
La requête suivante suit les stations où chaque vélo a commencé (dans `start_station_id`) et terminé (dans `end_station_id`) la journée du **25 octobre 2015**.  


In [11]:
# Requête pour suivre les stations de début et de fin le 25 octobre 2015, pour chaque vélo
start_end_query = """
                  SELECT bike_number,
                      TIME(start_date) AS trip_time,
                      FIRST_VALUE(start_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS first_station_id,
                      LAST_VALUE(end_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id,
                      start_station_id,
                      end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-10-25' 
                  """                 
# Exécuter la requête et renvoyer un DataFrame pandas
start_end_result = client.query(start_end_query).result().to_dataframe()
start_end_result.head()                 

bike_number trip_time  first_station_id  last_station_id  start_station_id  \
0          363  12:05:00                41               61                41   
1          363  13:56:00                41               61                54   
2          363  17:38:00                41               61                60   
3          363  22:17:00                41               61                46   
4          138  12:45:00                24               83                24   

   end_station_id  
0              54  
1              60  
2              46  
3              61  
4              83

La requête utilise à la fois `FIRST_VALUE()` et `LAST_VALUE()` comme fonctions analytiques.  

- La clause `PARTITION BY` divise les données en partitions basées sur la colonne `bike_number`. Comme cette colonne contient des identifiants uniques pour les vélos, cela garantit que les calculs sont effectués séparément pour chaque vélo.  
- La clause `ORDER BY` place les lignes dans chaque partition dans l'ordre chronologique.  
- Comme la clause de cadre de fenêtre est `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING`, pour chaque ligne, toute sa partition est utilisée pour effectuer le calcul. (Cela garantit que les valeurs calculées pour les lignes de la même partition sont identiques.)  

#### À vous de jouer  
Écrivez vos propres fonctions analytiques !  

Vous avez des questions ou des commentaires ? Visitez le forum de discussion du cours pour discuter avec d'autres apprenants.

---

# **EXERCICE**

### Introduction
Ici, vous utiliserez les Fonctions fenêtrées (*victoirefonctions de fenêtre) pour répondre à des questions sur l’ensemble de données **Trajets en taxi à Chicago**.

Avant de commencer, exécuter

In [18]:
from google.cloud import bigquery

# Créer un objet "Client"
client = bigquery.Client()

# Référencer le dataset "chicago_taxi_trips"
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# Requête API - récupérer le dataset
dataset = client.get_dataset(dataset_ref)

# Référencer la table "taxi_trips"
table_ref = dataset_ref.table("taxi_trips")

# Requête API - récupérer la table
table = client.get_table(table_ref)

# Afficher les cinq premières lignes de la table
client.list_rows(table, max_results=5).to_dataframe()


unique_key  \
0  5003bdd51918a9b5a52134096663b4d7e02395c5   
1  720534d264001b2644f682755b294067fdf1da21   
2  515b9a6d5234a4d1fb559dca27b3e1cc541a342d   
3  8afd48d966bf464e93127f43b72a8e405596a8eb   
4  fe73a44f01fdd2a3c740a01f53e97ec2ced93455   

                                             taxi_id  \
0  2130bc5fd239a4e3b304662424fb4cc7db0ca7abf78cc5...   
1  4bb55b69e710d1792f5fd4888001f4ff0ce34040f976f8...   
2  4bb55b69e710d1792f5fd4888001f4ff0ce34040f976f8...   
3  0150188f8c8e8973a198d4af0b427fca8ff48df2662d6b...   
4  0150188f8c8e8973a198d4af0b427fca8ff48df2662d6b...   

       trip_start_timestamp        trip_end_timestamp  trip_seconds  \
0 2013-02-25 14:15:00+00:00 2013-02-25 14:15:00+00:00           120   
1 2013-06-02 08:15:00+00:00 2013-06-02 08:15:00+00:00            60   
2 2013-06-24 06:15:00+00:00 2013-06-24 06:15:00+00:00            60   
3 2013-02-01 07:00:00+00:00 2013-02-02 00:00:00+00:00         61560   
4 2013-02-01 07:00:00+00:00 2013-02-02 00:00:00+00:00         60900   

   trip_miles  pickup_census_tract  dropoff_census_tract  \
0        0.00                 <NA>                  <NA>   
1        0.02                 <NA>                  <NA>   
2        0.04                 <NA>                  <NA>   
3        0.00                 <NA>                  <NA>   
4        0.00                 <NA>                  <NA>   

   pickup_community_area  dropoff_community_area  ...  extras  trip_total  \
0                   <NA>                    <NA>  ...     NaN         NaN   
1                   <NA>                    <NA>  ...     NaN         NaN   
2                      1                      77  ...     NaN         NaN   
3                   <NA>                    <NA>  ...     0.0         0.0   
4                   <NA>                    <NA>  ...     0.0         0.0   

   payment_type  company  pickup_latitude pickup_longitude  \
0          Cash     None              NaN              NaN   
1          Cash     None              NaN              NaN   
2          Cash     None        42.009623       -87.670167   
3          Cash     None              NaN              NaN   
4          Cash     None              NaN              NaN   

                            pickup_location  dropoff_latitude  \
0                                      None               NaN   
1                                      None               NaN   
2  POINT (-87.67016685690001 42.0096228806)         41.986712   
3                                      None               NaN   
4                                      None               NaN   

   dropoff_longitude                      dropoff_location  
0                NaN                                  None  
1                NaN                                  None  
2         -87.663416  POINT (-87.6634164054 41.9867117999)  
3                NaN                                  None  
4                NaN                                  None  

[5 rows x 23 columns]

---

### Question 1) Comment prédire la demande de taxis ?  
Imaginons que vous travaillez pour une entreprise de taxis et que vous souhaitez **prédire la demande de taxis**. Dans ce but, vous aimeriez créer un **graphique** montrant une **moyenne glissante** du nombre quotidien de trajets en taxi.  

Modifiez la requête **(partiellement complétée ci-dessous)** afin qu'elle renvoie un **DataFrame** avec **deux colonnes** :  

- **trip_date** : contient une entrée pour **chaque date** du **1er janvier 2016 au 31 mars 2016**.  
- **avg_num_trips** : affiche le **nombre moyen de trajets quotidiens**, calculé sur une **fenêtre mobile** comprenant la **date actuelle**, ainsi que les **3 jours précédents et les 3 jours suivants**, tant que ces jours restent dans la période des trois mois définis.  

Par exemple, pour calculer la valeur de cette colonne pour le **3 janvier 2016**, la fenêtre inclura :  
- le **nombre de trajets des 2 jours précédents** (1er et 2 janvier 2016),  
- la **date actuelle** (3 janvier 2016),  
- et les **3 jours suivants** (4, 5 et 6 janvier 2016).  

Cette requête est **partiellement complétée** pour vous, et vous devez **uniquement** écrire la partie qui calcule la colonne **avg_num_trips**.  

**Remarque** : Cette requête utilise une **expression de table commune** (*Common Table Expression - CTE*). Si vous avez besoin de réviser son fonctionnement, vous pouvez consulter ce [tutoriel](https://www.kaggle.com/learn/intro-to-sql) dans le cours **Introduction à SQL**.  


In [24]:
avg_num_trips_query = """
                        WITH trips_by_day AS
                        (
                            SELECT DATE(trip_start_timestamp) AS trip_date,
                                   COUNT(*) AS num_trips
                            FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                            WHERE trip_start_timestamp >= '2016-01-01' 
                                  AND trip_start_timestamp < '2016-04-01'
                            GROUP BY trip_date
                            
                        )
                        SELECT trip_date,
                               AVG(num_trips) 
                               OVER (
                                   ORDER BY trip_date
                                   ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                        FROM trips_by_day
                        ORDER BY trip_date
                     """
# Exécuter la requête et renvoyer un DataFrame pandas
avg_num_trips_result = client.query(avg_num_trips_query).result().to_dataframe()
avg_num_trips_result.head()


trip_date  avg_num_trips
0  2016-01-01   73313.750000
1  2016-01-02   72998.200000
2  2016-01-03   73452.166667
3  2016-01-04   74401.428571
4  2016-01-05   73767.285714

---

### Question 2) Pouvez-vous séparer et ordonner les trajets par zone communautaire ? 

La requête ci-dessous retourne un **DataFrame** avec trois colonnes issues de la table :  
- **pickup_community_area** (zone communautaire de départ),  
- **trip_start_timestamp** (horodatage du début du trajet),  
- **trip_end_timestamp** (horodatage de fin du trajet).  

Modifiez la requête pour ajouter une colonne supplémentaire appelée **trip_number**, qui affiche l'ordre dans lequel les trajets ont été effectués dans leurs zones communautaires respectives.  

Ainsi, le **premier trajet** de la journée provenant de la **zone communautaire 1** doit recevoir la valeur **1** ; le **deuxième trajet** de la journée depuis la **même zone** doit recevoir la valeur **2**, etc.  
De même, le **premier trajet** de la journée provenant de la **zone communautaire 2** doit recevoir la valeur **1**, et ainsi de suite.  

Notez qu'il existe plusieurs fonctions de numérotation permettant de résoudre ce problème (selon la façon dont vous souhaitez traiter les trajets qui ont commencé **exactement au même moment** dans la même zone communautaire). Pour répondre à cette question, veuillez utiliser la fonction **RANK()**.

In [29]:
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() OVER (
                            PARTITION BY pickup_community_area, DATE(trip_start_timestamp)
                            ORDER BY trip_start_timestamp
                        ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    ORDER BY pickup_community_area, trip_start_timestamp;
                    """
# Exécuter la requête et renvoyer un DataFrame pandas
trip_number_result = client.query(trip_number_query).result().to_dataframe()
trip_number_result.head()

pickup_community_area      trip_start_timestamp        trip_end_timestamp  \
0                   <NA> 2013-10-03 00:00:00+00:00 2013-10-03 00:00:00+00:00   
1                   <NA> 2013-10-03 00:00:00+00:00 2013-10-03 00:00:00+00:00   
2                   <NA> 2013-10-03 00:00:00+00:00 2013-10-03 00:15:00+00:00   
3                   <NA> 2013-10-03 00:00:00+00:00 2013-10-03 00:00:00+00:00   
4                   <NA> 2013-10-03 00:00:00+00:00 2013-10-03 00:00:00+00:00   

   trip_number  
0            1  
1            1  
2            1  
3            1  
4            1

### Explication :
- **RANK() OVER (...):** attribue un numéro de classement à chaque trajet.
- **PARTITION BY pickup_community_area, DATE(trip_start_timestamp):** classe les trajets par **zone communautaire** et par **jour**.
- **ORDER BY trip_start_timestamp:** classe les trajets par ordre chronologique.  
- **WHERE DATE(trip_start_timestamp) = '2013-10-03':** filtre les trajets pour la date spécifiée.  

Ainsi, chaque trajet est numéroté en fonction de son ordre d'apparition dans sa zone communautaire et pour chaque jour. 🚖📊

---

## Question 3) Combien de temps s'écoule entre les trajets ?

La (partielle) requête ci-dessous montre, pour chaque trajet dans la période sélectionnée, le **taxi_id**, le **trip_start_timestamp** et le **trip_end_timestamp** correspondants.

Votre tâche dans cet exercice est de modifier la requête pour inclure une colonne supplémentaire appelée **prev_break**, qui montre la durée de la pause (en minutes) que le chauffeur a eue avant chaque trajet (cela correspond au temps entre le **trip_start_timestamp** du trajet actuel et le **trip_end_timestamp** du trajet précédent). Partitionnez le calcul par **taxi_id**, et ordonnez les résultats au sein de chaque partition par **trip_start_timestamp**.

Certains résultats d'exemple sont montrés ci-dessous, où toutes les lignes correspondent au même chauffeur (ou **taxi_id**). Prenez le temps de vérifier que les valeurs dans la colonne **prev_break** ont du sens pour vous !

Notez que le premier trajet de la journée pour chaque chauffeur devrait avoir une valeur de **NaN** (not a number) dans la colonne **prev_break**.

In [34]:
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """
# Exécuter la requête et renvoyer un DataFrame pandas
break_time_result = client.query(break_time_query).result().to_dataframe()
break_time_result.head()

taxi_id  \
0  0daf144a0b4f020f8accbe2464c459e2a17b0d2e085d0b...   
1  0fdab9be71f6d88e3d3a2e115afc5a33d2bf74153792c5...   
2  18a877defd9c560103bd4c5b16d4de02031bfaa4eb859b...   
3  1e0f9d73609c308efb157811e55606b26b659aa59c5d63...   
4  1e0f9d73609c308efb157811e55606b26b659aa59c5d63...   

       trip_start_timestamp        trip_end_timestamp  prev_break  
0 2013-10-03 18:00:00+00:00 2013-10-03 18:00:00+00:00         285  
1 2013-10-03 19:45:00+00:00 2013-10-03 19:45:00+00:00         165  
2 2013-10-03 09:45:00+00:00 2013-10-03 09:45:00+00:00         510  
3 2013-10-03 10:30:00+00:00 2013-10-03 11:15:00+00:00         165  
4 2013-10-03 20:30:00+00:00 2013-10-03 21:15:00+00:00         555

Dans cette version :
- **LAG(trip_end_timestamp)** est utilisé pour obtenir le **trip_end_timestamp** du trajet précédent pour chaque chauffeur (**taxi_id**).
- **PARTITION BY taxi_id** permet de calculer la différence entre les trajets pour chaque chauffeur.
- **ORDER BY trip_start_timestamp** garantit que les trajets sont bien triés par heure de départ.